<a href="https://colab.research.google.com/github/thetinybug/MLDL-LearnPyTorch/blob/master/Pytorch_on_TPU%2C_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Choose Mode

In [11]:
use_tpu = True #@param {type:"boolean"}
use_gpu = True #@param {type:"boolean"}

In [12]:
import os
tpu_existed = True

try:
    assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
    print("TPU Ready!")
except KeyError:
    print("Not Exist TPU!")
    tpu_existed = False

Not Exist TPU!


# Install PyTorch

In [13]:
!pip3 install torchvision

In [14]:
if use_tpu and tpu_existed:
    VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

# Import PyTorch

In [15]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

if use_tpu and tpu_existed:
    # imports the torch_xla package
    import torch_xla
    import torch_xla.core.xla_model as xm

# Initialize Hyper-parameters

In [16]:
input_size    = 784   # The image size = 28 x 28 = 784
hidden_size   = 500   # The number of nodes at the hidden layer
num_classes   = 10    # The number of output classes. In this case, from 0 to 9
num_epochs    = 5     # The number of times entire dataset is trained
batch_size    = 100   # The size of input data took for one iteration
learning_rate = 1e-3  # The speed of convergence

# Download MNIST Dataset

MNIST is a huge database of handwritten digits (i.e. 0 to 9) that is often used in image classification.

In [17]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Build the Feedforward Neural Network

### Feedforward Neural Network Model Structure

The FNN includes two fully-connected layers (i.e. fc1 & fc2) and a non-linear ReLU layer in between. Normally we call this structure **1-hidden layer FNN**, without counting the output layer (fc2) in.

By running the forward pass, the input images (x) can go through the neural network and generate a output (out) demonstrating how are the likabilities it belongs to each of the 10 classes. _For example, a cat image can have 0.8 likability to a dog class and a 0.3 likability to a airplane class._

In [18]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Instantiate the FNN

We now create a real FNN based on our structure.

In [19]:
net = Net(input_size, hidden_size, num_classes)

# Choose the Loss Function and Optimizer

Loss function (**criterion**) decides how the output can be compared to a class, which determines how good or bad the neural network performs. And the **optimizer** chooses a way to update the weight in order to converge to find the best weights in this neural network.

In [10]:
# net = Net(input_size, hidden_size, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Training the FNN Model

This process might take around 3 to 5 minutes depending on your machine. The detailed explanations are listed as comments (#) in the following codes.

In [11]:
import timeit
print(100*'=')
net = Net(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

if use_tpu and tpu_existed:
    dev = xm.xla_device()
    # print(" Device created!")
    net = net.to(dev)
    print("Use TPU")
elif use_gpu and torch.cuda.is_available():
    net.cuda()
    print("Use GPU")
else:
    print("Use CPU")
print(100*'-')

for epoch in range(num_epochs):
    start = timeit.default_timer()
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        if use_tpu and tpu_existed:
            images = images.to(dev)
            labels = labels.to(dev)
        elif use_gpu and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data))
    runtime = timeit.default_timer() - start
    print("Epoch Runtime: ", runtime)

Use TPU
----------------------------------------------------------------------------------------------------
Epoch [1/5], Step [100/600], Loss: 2.3074
Epoch [1/5], Step [200/600], Loss: 2.3331
Epoch [1/5], Step [300/600], Loss: 2.3297
Epoch [1/5], Step [400/600], Loss: 2.3268
Epoch [1/5], Step [500/600], Loss: 2.3279
Epoch [1/5], Step [600/600], Loss: 2.3343
Epoch Runtime:  9.8889253440002
Epoch [2/5], Step [100/600], Loss: 2.3344
Epoch [2/5], Step [200/600], Loss: 2.3308
Epoch [2/5], Step [300/600], Loss: 2.3271
Epoch [2/5], Step [400/600], Loss: 2.3350
Epoch [2/5], Step [500/600], Loss: 2.3412
Epoch [2/5], Step [600/600], Loss: 2.3176
Epoch Runtime:  9.288507456999923
Epoch [3/5], Step [100/600], Loss: 2.3168
Epoch [3/5], Step [200/600], Loss: 2.3304
Epoch [3/5], Step [300/600], Loss: 2.3135
Epoch [3/5], Step [400/600], Loss: 2.3418
Epoch [3/5], Step [500/600], Loss: 2.3128
Epoch [3/5], Step [600/600], Loss: 2.3252
Epoch Runtime:  9.270580026000061
Epoch [4/5], Step [100/600], Loss: 

In [20]:
import timeit
print(100*'=')
net = Net(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
# if use_tpu and tpu_existed:
#     dev = xm.xla_device()
#     print(" Device created!")
#     net = net.to(dev)
#     print("Use TPU")
if use_gpu and torch.cuda.is_available():
    net.cuda()
    print("Use GPU")
else:
    print("Use CPU")
print(100*'-')

for epoch in range(num_epochs):
    start = timeit.default_timer()
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        # if use_tpu and tpu_existed:
        #     images = images.to(dev)
        #     labels = labels.to(dev)
        if use_gpu and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data))
    runtime = timeit.default_timer() - start
    print("Epoch Runtime: ", runtime)

Use GPU
----------------------------------------------------------------------------------------------------
Epoch [1/5], Step [100/600], Loss: 0.5069
Epoch [1/5], Step [200/600], Loss: 0.3709
Epoch [1/5], Step [300/600], Loss: 0.2546
Epoch [1/5], Step [400/600], Loss: 0.2411
Epoch [1/5], Step [500/600], Loss: 0.0745
Epoch [1/5], Step [600/600], Loss: 0.1457
Epoch Runtime:  7.717695975999959
Epoch [2/5], Step [100/600], Loss: 0.1001
Epoch [2/5], Step [200/600], Loss: 0.0940
Epoch [2/5], Step [300/600], Loss: 0.0936
Epoch [2/5], Step [400/600], Loss: 0.1156
Epoch [2/5], Step [500/600], Loss: 0.0560
Epoch [2/5], Step [600/600], Loss: 0.0754
Epoch Runtime:  7.4145907609999995
Epoch [3/5], Step [100/600], Loss: 0.0613
Epoch [3/5], Step [200/600], Loss: 0.1479
Epoch [3/5], Step [300/600], Loss: 0.0356
Epoch [3/5], Step [400/600], Loss: 0.0603
Epoch [3/5], Step [500/600], Loss: 0.0336
Epoch [3/5], Step [600/600], Loss: 0.0215
Epoch Runtime:  7.345578411999895
Epoch [4/5], Step [100/600], Los

#NOT USE

## Testing the FNN Model

Similar to training the neural network, we also need to load batches of test images and collect the outputs. The differences are that:

1. No loss & weights calculation
2. No wights update
3. Has correct prediction calculation


In [ ]:
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, 28*28))
    
#     if use_cuda and torch.cuda.is_available():
#         images = images.cuda()
#         labels = labels.cuda()
    
    
#     outputs = net(images)
#     _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
#     total += labels.size(0)                    # Increment the total count
#     correct += (predicted == labels).sum()     # Increment the correct count
    
# print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %


## Save the trained FNN Model for future use

We save the trained model as a pickle that can be loaded and used later.

In [ ]:
# torch.save(net.state_dict(), 'fnn_model.pkl')

## Congrats

You have done building your first Feedforward Neural Network!